# Prediction

###### Imports

In [2]:
# This Python file uses the following encoding: utf-8
from lib.words import not_so_naive_split, reduce_dictionary, sentence_vector, tokenize_doc
from lib.pretty_testing import predict_test
from keras.models import model_from_json
from keras.preprocessing import sequence
from lib.untar import ExtractNested
from gensim.models import Doc2Vec
import lib.text_extraction as te
from functools import partial
from lib.utils import save_csv

import numpy as np
import codecs
import json
import os
import sys

###### Global Variables

In [3]:
pdf_names = ['../atti_costitutivi/5122462300001.pdf',
             '../files_to_predict/T3LAB-INJENIA_Analisi Preliminare.pdf',
             '../files_to_predict/5 Planning Robot.pdf',
             '../atti_costitutivi/5122464750001.pdf']

labels_map = ['NON COSTITUTIVO', 'COSTITUTIVO']

tar_root = '../prova.tar.gz'
pdf_folder = '../files_to_predict'
png_dir = '../tmp'

min_words = 150
pages = 5

###### Functions

In [8]:
# Accetto solo file con dimensione < 5MB
def file_filter(f):
    return os.path.getsize(str(f.absolute()))<(1024**2*5)
    #return f.name.endswith(u'001.pdf') and os.path.getsize(str(f.absolute()))<(1024**2*5)

def embed_document(model, doc, permitted_words):
    return [sentence_vector(model, sentence, permitted_words) for sentence in doc]

def predict_documents_str(filenames, txts, gensim_model, keras_model, permitted_words):
    filtered_filenames = [f for f,t in zip(filenames, txts) if (t != None and len(t)>0)]
    not_empty_txts =  [t for t in txts if  (t != None and len(t)>0)]
    
    splitted_txts = [tokenize_doc(txt) for txt in not_empty_txts] 
    filtered_txts = [list(reduce_dictionary(document, permitted_words)) for document in splitted_txts]
    embedded_txts = [embed_document(gensim_model, doc, permitted_words) for doc in filtered_txts]
    padded_data = sequence.pad_sequences(embedded_txts, maxlen=200, padding="pre", truncating="post", value=0.0, dtype='float32')
    probs = keras_model.predict_proba(padded_data, verbose=0)
    return [prob[0] for prob in probs], filtered_filenames

def predict_documents_pdf(filenames, gensim_model, keras_model, permitted_words, do_ocr=False):
    txts = [te.extract_text(filename, do_ocr, png_dir, min_words, pages) for filename in filenames]
    return predict_documents_str(filenames, txts, gensim_model, keras_model, permitted_words)

def predict_documents_txt(filenames, gensim_model, keras_model, permitted_words):
    txts = [open(filename).read() for filename in filenames]
    return predict_documents_str(filenames, txts, gensim_model, keras_model, permitted_words)

###### Loading models

In [5]:
models = {}
models['gensim_model'] = Doc2Vec.load('models/gensim_model_5000.d2v')

with open('models/keras_model.json') as f:
    models['keras_model'] = model_from_json(f.read())
models['keras_model'].load_weights("models/keras_weights_5000.h5")

with open('first_5000_words.json') as f:
    models['permitted_words'] = set(json.load(f))
    
#predict_pdfs = partial(predict_documents_pdf, gensim_model=gensim_model, keras_model=keras_model, permitted_words=permitted_words)
predict_pdfs = partial(predict_documents_pdf, **models)

###### Tar extraction (if needed)

In [10]:
# Solo se si deve partire da dei tar
ExtractNested(tar_root)

Extracting '../prova.tar.gz' to 'prova'... Done!


In [12]:
move_flattened_files(tar_root[:-7], pdf_folder, file_filter)

In [5]:
pdf_names = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder)]

### Prediction

###### With pdf as source

In [9]:
predictions, filtered_filenames = predict_pdfs(pdf_names)

In [10]:
for name, pred in zip(filtered_filenames, predictions):
    print os.path.basename(name)[:50].ljust(50), '\t-\t', labels_map[int(round(pred))], ' (%f)' % pred

5122462300001.pdf                                  	-	COSTITUTIVO  (0.991282)
T3LAB-INJENIA_Analisi Preliminare.pdf              	-	NON COSTITUTIVO  (0.004446)
5 Planning Robot.pdf                               	-	NON COSTITUTIVO  (0.003736)
5122464750001.pdf                                  	-	COSTITUTIVO  (0.998065)


In [8]:
def basenames(files):
    return [os.path.basename(f) for f in files]

filt_filenames_set = set(filtered_filenames)
labels = [labels_map[int(round(pred))] for pred in predictions]
results = list(zip(basenames(filtered_filenames), labels, predictions))

not_predicted_pdfs = [pdf for pdf in pdf_names if pdf not in filt_filenames_set]
not_predicted_results = ['']*(len(pdf_names)-len(filtered_filenames))
all_results = results + list(zip(basenames(not_predicted_pdfs), not_predicted_results, not_predicted_results))
headed_results = [('Nome file','Predizione','Output rete')] + all_results

save_csv(headed_results, '../predictions.csv')

###### With txt as source

In [12]:
txt_names = ['../ocr/scanned_non_costitutivi/3591900710001.txt']

pred2, filtered_filenames = predict_documents_txt(txt_names, **models)

In [13]:
for name, pred in zip(filtered_filenames, pred2):
    print os.path.basename(name)[:50].ljust(50), '\t-\t', labels_map[int(round(pred))], ' (%f)' % pred

3591900710001.txt                                  	-	NON COSTITUTIVO  (0.043717)
